# Basic imports

In [36]:
import trackhhl.toy.simple_generator as toy
import trackhhl.hamiltonians.simple_hamiltonian as hamiltonian
import numpy as np

# Detector

We are going to define a simple detector geometry of 3 parallel infinitely large modules, placed along the $z$-axis

In [37]:
N_MODULES = 3
LX = float("+inf")
LY = float("+inf")
Z_SPACING = 1.0

detector = toy.SimpleDetectorGeometry(
    module_id=list(range(N_MODULES)),
    lx=[LX]*N_MODULES,
    ly=[LY]*N_MODULES,
    z=[i+Z_SPACING for i in range(N_MODULES)]
)

detector

SimpleDetectorGeometry(module_id=[0, 1, 2], lx=[inf, inf, inf], ly=[inf, inf, inf], z=[1.0, 2.0, 3.0])

# Particle generator

Now we define a simple particle generator that will fire particles flying in straight lines through the our detector.

In [38]:
generator = toy.SimpleGenerator(
    detector_geometry=detector,
    theta_max=np.pi/6
)

Let's generate a simple event

In [39]:
N_PARTICLES = 3
event = generator.generate_event(N_PARTICLES)
event.hits

[Hit(hit_id=0, x=0.2404342221839211, y=0.20324647875725627, z=1.0, module_id=0, track_id=0),
 Hit(hit_id=3, x=0.11508383276062542, y=-0.2668605495867298, z=1.0, module_id=0, track_id=1),
 Hit(hit_id=6, x=0.025296548983003105, y=0.15114944451347884, z=1.0, module_id=0, track_id=2),
 Hit(hit_id=1, x=0.4808684443678422, y=0.40649295751451253, z=2.0, module_id=1, track_id=0),
 Hit(hit_id=4, x=0.23016766552125084, y=-0.5337210991734596, z=2.0, module_id=1, track_id=1),
 Hit(hit_id=7, x=0.05059309796600621, y=0.3022988890269577, z=2.0, module_id=1, track_id=2),
 Hit(hit_id=2, x=0.7213026665517632, y=0.6097394362717687, z=3.0, module_id=2, track_id=0),
 Hit(hit_id=5, x=0.34525149828187623, y=-0.8005816487601893, z=3.0, module_id=2, track_id=1),
 Hit(hit_id=8, x=0.0758896469490093, y=0.45344833354043645, z=3.0, module_id=2, track_id=2)]

# Hamiltonian initialization

Let's initialize the Hamiltonian.

In [40]:
ham = hamiltonian.SimpleHamiltonian(
    epsilon=1e-7,
    gamma=2.0,
    delta=1.0
)

In [41]:
ham.construct_hamiltonian(event=event)

(<18x18 sparse matrix of type '<class 'numpy.float64'>'
 	with 24 stored elements in Compressed Sparse Column format>,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.]))

# Classical solver

Let's solve this event using a classical linear solver (the Conjugate Gradient Descent method)

In [42]:
classical_solution = ham.solve_classicaly()
classical_solution

array([0.5       , 0.33333333, 0.33333333, 0.33333333, 0.5       ,
       0.33333333, 0.33333333, 0.33333333, 0.5       , 0.5       ,
       0.33333333, 0.33333333, 0.33333333, 0.5       , 0.33333333,
       0.33333333, 0.33333333, 0.5       ])

Let's apply a threshold $T = 0.45$

In [43]:
T = .45

In [44]:
discretized_classical_solution = (classical_solution > T).astype(int)
discretized_classical_solution

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1])

# HHL solver

Now let's solve the same event using the HHL linear solver provided by Qiskit.

In [45]:
hhl_solution = ham.solve_hhl()
hhl_solution

/Users/davidenicotra/Library/Caches/pypoetry/virtualenvs/trackhhl-oT22-ukJ-py3.11/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/davidenicotra/Library/Caches/pypoetry/virtualenvs/trackhhl-oT22-ukJ-py3.11/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


array([0.5       , 0.33333333, 0.33333333, 0.33333333, 0.5       ,
       0.33333333, 0.33333333, 0.33333333, 0.5       , 0.5       ,
       0.33333333, 0.33333333, 0.33333333, 0.5       , 0.33333333,
       0.33333333, 0.33333333, 0.5       ])

Let's discretize using the same threshold $T=0.45$

In [46]:
discretized_hhl_solution = (hhl_solution > .45).astype(int)
discretized_classical_solution

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1])

# Comparison

Let's compare the two solutions with the Monte Carlo truth

In [47]:
print("Classical solution: \t", discretized_classical_solution)
print("HHL solution: \t\t", discretized_classical_solution)
print()
MC_truth = [ 1 if seg.hit_from.track_id == seg.hit_to.track_id else 0 for seg in ham.segments]
print("MC truth: \t\t",np.array(MC_truth))

Classical solution: 	 [1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1]
HHL solution: 		 [1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1]

MC truth: 		 [1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1]
